## Summarizing top posts specific to an APT#

### 1. extract data for each APT

In [1]:
import pandas as pd
from os import listdir

path = '../data/'
files = listdir('../data/')
df = pd.DataFrame(columns=["url", "query", "text"])

for f in files:
    temp = pd.read_csv(path + f)
    df = df.append(temp)

# aptNum = "APT%s" % n
# df.loc[df['query'] == 'APT1']
df[df['query'].isin(['APT1'])]
apt_json = 'APT1'+'.json'
df.to_json(apt_json, orient='records')

df
# df.iloc[0:0]

/home/tamanna/.local/lib/python2.7/site-packages/pandas/core/frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


,Unnamed: 0,query,text,url
0,0,APT10,"Introduction\nIn April 2017, a previously unkn...",https://www.crowdstrike.com/blog/two-birds-one...
0,0,APT10,APT10 Background APT10 (MenuPass Group) is a ...,https://www.fireeye.com//blog/threat-research/...
1,1,APT10,"Introduction In July 2018, FireEye devices de...",https://www.fireeye.com//blog/threat-research/...
0,0,APT10,China-linked APT10 Hackers Update Attack Techn...,https://www.securityweek.com/china-linked-apt1...
1,1,APT10,China-Linked Group Uses New Malware in Japan A...,https://www.securityweek.com/china-linked-grou...
2,2,APT10,Operation Cloud Hopper: China-based Hackers Ta...,https://www.securityweek.com/operation-cloud-h...
3,3,APT10,Poison Ivy RAT Campaign Leverages New Delivery...,https://www.securityweek.com/poison-ivy-rat-ca...
4,4,APT10,China-Linked Hackers Target U.S. Trade Group\r...,https://www.securityweek.com/china-linked-hack...
5,5,APT10,DHS Warns of Attacks on Managed Service Provid...,https://www.securityweek.com/dhs-warns-attacks...
6,6,APT10,Don't Fall Victim to IP Theft and Corporate Es...,https://www.securityweek.com/dont-fall-victim-...


In [2]:
import pandas as pd
from os import listdir

path = '../data/'
files = listdir('../data/')
df = pd.DataFrame(columns=["url", "query", "text"])

aptNumbers = [1, 3, 5, 10, 12, 15, 16, 17, 18, 19, 27, 28, 29, 30, 32, 33, 34, 35, 37, 38]

for n in aptNumbers:
    for f in files:
        temp = pd.read_csv(path + f)
        df = df.append(temp)
    aptNum = "APT%s" % n
    df=df.loc[df['query'] == aptNum]
    apt_json = aptNum+'.json'
    df.to_json(apt_json, orient='records')
#     df.iloc[0:0]

### 2. summarizing the posts

In [ ]:
import json
import nltk
import numpy
import sys

BLOG_DATA = "APT38.json"

N = 100  # Number of words to consider
CLUSTER_THRESHOLD = 5  # Distance between words to consider
TOP_SENTENCES = 5  # Number of sentences to return for a "top n" summary

# Approach taken from "The Automatic Creation of Literature Abstracts" by H.P. Luhn

def _score_sentences(sentences, important_words):
    scores = []
    sentence_idx = -1

    for s in [nltk.tokenize.word_tokenize(s) for s in sentences]:

        sentence_idx += 1
        word_idx = []

        # For each word in the word list...
        for w in important_words:
            try:
                # Compute an index for where any important words occur in the sentence.

                word_idx.append(s.index(w))
            except ValueError, e: # w not in this particular sentence
                pass

        word_idx.sort()

        # It is possible that some sentences may not contain any important words at all.
        if len(word_idx)== 0: continue

        # Using the word index, compute clusters by using a max distance threshold
        # for any two consecutive words.

        clusters = []
        cluster = [word_idx[0]]
        i = 1
        while i < len(word_idx):
            if word_idx[i] - word_idx[i - 1] < CLUSTER_THRESHOLD:
                cluster.append(word_idx[i])
            else:
                clusters.append(cluster[:])
                cluster = [word_idx[i]]
            i += 1
        clusters.append(cluster)

        # Score each cluster. The max score for any given cluster is the score 
        # for the sentence.

        max_cluster_score = 0
        for c in clusters:
            significant_words_in_cluster = len(c)
            total_words_in_cluster = c[-1] - c[0] + 1
            score = 1.0 * significant_words_in_cluster \
                * significant_words_in_cluster / total_words_in_cluster

            if score > max_cluster_score:
                max_cluster_score = score

        scores.append((sentence_idx, score))

    return scores

def summarize(txt):
    sentences = [s for s in nltk.tokenize.sent_tokenize(txt)]
    normalized_sentences = [s.lower() for s in sentences]

    words = [w.lower() for sentence in normalized_sentences for w in
             nltk.tokenize.word_tokenize(sentence)]

    fdist = nltk.FreqDist(words)

    top_n_words = [w[0] for w in fdist.items() 
            if w[0] not in nltk.corpus.stopwords.words('english')][:N]
    
#     print "_______________"
#     print top_n_words
#     print "_______________"

    scored_sentences = _score_sentences(normalized_sentences, top_n_words)

    # Summarization Approach 1:
    # Filter out nonsignificant sentences by using the average score plus a
    # fraction of the std dev as a filter

    avg = numpy.mean([s[1] for s in scored_sentences])
    std = numpy.std([s[1] for s in scored_sentences])
    mean_scored = [(sent_idx, score) for (sent_idx, score) in scored_sentences
                   if score > avg + 0.5 * std]

    # Summarization Approach 2:
    # Another approach would be to return only the top N ranked sentences

    top_n_scored = sorted(scored_sentences, key=lambda s: s[1])[-TOP_SENTENCES:]
    top_n_scored = sorted(top_n_scored, key=lambda s: s[0])

    # Decorate the post object with summaries

    return dict(top_n_summary=[sentences[idx] for (idx, score) in top_n_scored],
                mean_scored_summary=[sentences[idx] for (idx, score) in mean_scored])

blog_data = json.loads(open(BLOG_DATA).read())

post_count = 0
f = open('APT38-summary.txt','wt')
sys.stdout = open('APT38-summary.txt','wt')
for post in blog_data:
       
    post.update(summarize(post['text']))

    post_count+=1
#     break
    print 'Post',post_count,'.......................................................................'
    print post['url']
    print 
    print 'Top N Summary'
    print '-------------'
    print (' '.join(post['top_n_summary'])).encode('ascii', 'ignore').decode('ascii')
    print 
    print 'Mean Scored Summary'
    print '-------------------'
    print (' '.join(post['mean_scored_summary'])).encode('ascii', 'ignore').decode('ascii')
    print 
    
f.close()
sys.stdout.close()